### Import Libraries

In [5]:
from pyspark.sql.functions import col, from_json, upper

from lib.database import get_postgres_table
from lib.spark import get_spark_session

### Configure Session

In [2]:
spark = get_spark_session()
sc = spark.sparkContext

### Get Tables

In [26]:
actions = get_postgres_table(database="activate_rule_production",
                             port=9037,
                             table="actions")

rule_fulfillments = get_postgres_table(database="activate_rule_production",
                                       partition_col="created_at",
                                       port=9037,
                                       table="rule_fulfillments")

transactions = get_postgres_table(database="activate_core_production",
                                  partition_col="created_at",
                                  port=9037,
                                  table="transactions")

transaction_line_items = get_postgres_table(database="activate_core_production",
                                            partition_col="created_at",
                                            port=9037,
                                            table="transaction_line_items")

### Working Code

In [27]:
actions = actions.where(upper(col("type")).contains("DISCOUNT"))
action_configurations = spark.read.json(actions.select("configuration").rdd.map(lambda x: x[0]))
actions = actions.withColumn("configuration", from_json("configuration", action_configurations.schema)) \
                 .select(col("id").alias("action_id"),
                         col("type").alias("action_type"),
                         "rule_id",
                         col("created_at").alias("action_created_at"),
                         col("updated_at").alias("action_updated_at"),
                         col("deleted_at").alias("action_deleted_at"),
                         "offer_component_type",
                         "offer_component_id",
                         "configuration.*")

rule_fulfillments = rule_fulfillments.where("upper(source_type) = 'TRANSACTION'") \
                                     .select(col("id").alias("rule_fulfillment_id"),
                                             "rule_id",
                                             col("membership_id").alias("rule_fulfillment_membership_id"),
                                             col("count").alias("rule_fulfillment_count"),
                                             col("created_at").alias("rule_fulfillment_created_at"),
                                             col("updated_at").alias("rule_fulfillment_updated_at"),
                                             col("source_id").alias("transaction_id"),
                                             col("deleted_at").alias("rule_fulfillment_deleted_at"))

transactions = transactions.withColumnRenamed("id", "transaction_id") \
                           .drop("created_at", "updated_at", "deleted_at", "transaction_time_at")
transaction_line_items = transaction_line_items.withColumnRenamed("id", "line_item_id") \
                                               .withColumnRenamed("money_amount", "line_item_money_amount") \
                                               .withColumnRenamed("quantity", "line_item_quantity") \
                                               .drop("created_at", "updated_at", "deleted_at")
transactions = transactions.join(transaction_line_items, ["transaction_id", "membership_id"])

In [30]:
df = rule_fulfillments.join(actions, ["rule_id"]) \
                      .join(transactions, ["transaction_id"])

In [31]:
display(df.to_pandas_on_spark())

,transaction_id,rule_id,rule_fulfillment_id,rule_fulfillment_membership_id,rule_fulfillment_count,rule_fulfillment_created_at,rule_fulfillment_updated_at,rule_fulfillment_deleted_at,action_id,action_type,action_created_at,action_updated_at,action_deleted_at,offer_component_type,offer_component_id,award_per,count_groups,discount_threshold,discount_type,discounted_product_counter,effect_grouping_configuration,money_amount,money_discount,percent_discount,purchase_amount_threshold,purchase_threshold,membership_id,location_id,money_currency_code,request_id,partner_id,custom_data,processed_at,external_location_id,external_transaction_id,line_item_id,sku,promo_code,line_item_quantity,line_item_money_amount,upc,group,line_item_tags,product_groups,transaction_time_at,fuel_grade_id
0,0001482f-ece2-4d78-8f57-fb84826bf455,cb034e3b-cbde-49ba-a9e8-ac9102e4e694,84beaed5-5632-4002-a834-505ff0fe8ad1,37643010-e344-4d33-a17b-095038eddb48,1,2021-12-22 19:31:13.925955,2021-12-22 19:31:13.925955,NaT,a0d658de-6457-4517-8b50-22ed91f60c78,Actions::Discount,2021-12-09 22:11:44.356575,2021-12-09 22:11:44.356575,NaT,EarningMechanism,5ec5b5fe-250e-4a0d-8008-1f59d7f4df94,per_unit,None,0.0,price_override,None,per_unit,0.0,NaN,0.0,NaN,1.0,37643010-e344-4d33-a17b-095038eddb48,None,None,a983ab32-fea1-4fb2-85a8-e783fc905b25,None,{},2021-12-22 19:31:49.189572,88b3b102-6d01-493a-991f-a10351458694,0101235141,45ec4d03-64b9-4591-bd56-222d84d89c17,mars ss 3 msktrs,None,1.000,0,4000042208,57,None,"[""027acf92-e806-450a-90a8-aedd3fc0dbba"", ""3f35...",2021-12-22 19:27:16,None
1,0001482f-ece2-4d78-8f57-fb84826bf455,cb034e3b-cbde-49ba-a9e8-ac9102e4e694,84beaed5-5632-4002-a834-505ff0fe8ad1,37643010-e344-4d33-a17b-095038eddb48,1,2021-12-22 19:31:13.925955,2021-12-22 19:31:13.925955,NaT,a0d658de-6457-4517-8b50-22ed91f60c78,Actions::Discount,2021-12-09 22:11:44.356575,2021-12-09 22:11:44.356575,NaT,EarningMechanism,5ec5b5fe-250e-4a0d-8008-1f59d7f4df94,per_unit,None,0.0,price_override,None,per_unit,0.0,NaN,0.0,NaN,1.0,37643010-e344-4d33-a17b-095038eddb48,None,None,a983ab32-fea1-4fb2-85a8-e783fc905b25,None,{},2021-12-22 19:31:49.189572,88b3b102-6d01-493a-991f-a10351458694,0101235141,3626a455-c877-4159-9ec9-1650b1d9c3d1,cef ftn lrg,None,1.000,0,502,7,None,"[""069deb75-910e-4ab1-ad1a-7a8c558e6510"", ""84d3...",2021-12-22 19:27:16,None
2,0001482f-ece2-4d78-8f57-fb84826bf455,cb034e3b-cbde-49ba-a9e8-ac9102e4e694,84beaed5-5632-4002-a834-505ff0fe8ad1,37643010-e344-4d33-a17b-095038eddb48,1,2021-12-22 19:31:13.925955,2021-12-22 19:31:13.925955,NaT,a0d658de-6457-4517-8b50-22ed91f60c78,Actions::Discount,2021-12-09 22:11:44.356575,2021-12-09 22:11:44.356575,NaT,EarningMechanism,5ec5b5fe-250e-4a0d-8008-1f59d7f4df94,per_unit,None,0.0,price_override,None,per_unit,0.0,NaN,0.0,NaN,1.0,37643010-e344-4d33-a17b-095038eddb48,None,None,a983ab32-fea1-4fb2-85a8-e783fc905b25,None,{},2021-12-22 19:31:49.189572,88b3b102-6d01-493a-991f-a10351458694,0101235141,a3a3d7fc-5e93-48da-ae5b-803815734eae,blk n mld orig uprt,None,1.000,105,7013710018,1,None,"[""24dfd204-bde2-4523-bcd6-1329669cac55"", ""3f35...",2021-12-22 19:27:16,None
3,0001482f-ece2-4d78-8f57-fb84826bf455,d65bc88c-59bd-4ab3-9f21-f5800f0c0a76,20611239-d098-4107-9350-6d2223c7bbeb,37643010-e344-4d33-a17b-095038eddb48,1,2021-12-22 19:31:13.950807,2021-12-22 19:31:13.950807,NaT,d102135e-0bea-40fd-a7d7-1d15f25c803d,Actions::Discount,2021-12-10 18:24:25.192992,2021-12-10 18:24:25.192992,NaT,EarningMechanism,613ebe04-51d0-47d6-b2cb-97a0ec39ccde,per_unit,None,0.0,price_override,None,per_unit,0.0,NaN,0.0,NaN,1.0,37643010-e344-4d33-a17b-095038eddb48,None,None,a983ab32-fea1-4fb2-85a8-e783fc905b25,None,{},2021-12-22 19:31:49.189572,88b3b102-6d01-493a-991f-a10351458694,0101235141,45ec4d03-64b9-4591-bd56-222d84d89c17,mars ss 3 msktrs,None,1.000,0,4000042208,57,None,"[""027acf92-e806-450a-90a8-aedd3fc0dbba"", ""3f35...",2021-12-22 19:27:16,None
4,0001482f-ece2-4d78-8f57-fb84826bf455,d65bc88c-59bd-4ab3-9f21-f5800f0c0a76,20611239-d098-4107-9350-6d2223c7bbeb,37643